In [177]:
from scipy.stats import norm
import torch.distributions as tdist
import numpy as np

sample_norm = norm(
    loc=13,
    scale=13 ** .7
).cdf(18)

print(sample_norm)
random_noise = np.random.uniform(-0.06, 0.01, 1).round(3)
print(random_noise.mean().round(3))

0.7968016923715895
0.001


In [75]:
random_noise.mean()

-3.1373410196329804e-05

In [ ]:
sample_dist.

In [26]:
s = np.random.normal(27, 15, 10000)



In [27]:
s.mean()

26.966019494179836

In [1]:
!pip install python-dotenv --quiet
!pip install gym stable-baselines3[extra] awscli boto3 pqdm awscli --quiet


In [2]:
%load_ext dotenv
%dotenv env

In [3]:
!aws s3 sync s3://dissertation-data-dmiller/rl_ready_data_conv rl_ready_data_conv --quiet

In [4]:
# %load rl_constant.py
# %load rl_constant.py
FEATURE_COLUMNS = [
    "country_count", 
    "date_hour_sin", 
    "date_hour_cos",
    "date_minute_sin",
    "date_minute_cos",
    
    "session_30_count",
    "session_5_count",
    "cum_session_event",
    "cum_session_time",
    "expanding_click_average",
   
    "cum_platform_time",
    "cum_platform_event",
    "cum_projects",
    "average_event_time",
    "delta_last_event",
    
    "rolling_session_time",
    "rolling_session_events",
    "rolling_session_gap",
    "previous_session_time",
    "previous_session_events",
]

METADATA = [
    "user_id",
    "session_30_count_raw",
    "cum_platform_event_raw",
    "cum_platform_time_raw",
    "cum_session_time_raw",
    "cum_session_event_raw",
    "date_time"
]

RL_STAT_COLS = [
    'session_size',
    'session_minutes',
    'sim_size',
    'sim_minutes',
    'reward'
]

PREDICTION_COLS = [
    "seq_40",
    "session_terminates_30_minutes"
    # "label"
]

LOAD_COLS = list(set(FEATURE_COLUMNS + METADATA + RL_STAT_COLS + PREDICTION_COLS))

In [5]:
# %load environment
import gym
import numpy as np
from scipy.stats import norm

import numpy as np
from scipy.stats import norm 
import gym
from datetime import datetime
from copy import deepcopy

class CitizenScienceEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, dataset, out_features, n_sequences, evaluation=False):
        """
        trajectories: dictionary of user_id to their respective trajectories.
        n_sequences: number of sequences used for preprocessing.
        n_features: number of features used for preprocessing.
        """
        super(CitizenScienceEnv, self).__init__()
        self.dataset = dataset
        self.n_sequences = n_sequences
        self.current_session = None
        self.current_session_index = 0
        self.reward = 0
        self.n_sequences = n_sequences
        self.out_features = out_features
        
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(len(out_features), n_sequences + 1), dtype=np.float32)
        self.evalution = evaluation
        self.episode_bins = []
        self.exp_runs = 0

    def reset(self):
        user_to_run, session_to_run = self.dataset.sample(1)[['user_id', 'session_30_count_raw']].values[0]
        self.current_session = self._get_events(user_to_run, session_to_run)
        self.metadata = self._metadata()
        self.current_session_index = 0
        self.reward = 0
        return self._state()
    
    def _row_to_dict(self, metadata):
        """
        Convert a row of metadata to a dictionary.
        """
        return metadata.to_dict()

    def step(self, action):
        
        self._take_action(action)
        next_state, done, meta = self._calculate_next_state()
        
        if done:
            current_session_index = self.current_session_index if \
                self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
            
            self.exp_runs += 1
        
            self.metadata['ended'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
            self.metadata['reward'] = self.current_session.iloc[current_session_index]['reward']
            self.metadata['session_exp_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            self.metadata['exp_runs'] = self.exp_runs
            self.episode_bins.append(self._row_to_dict(self.metadata))
            
            return next_state, float(self.reward), done, {}
        else:
            self.reward = self.current_session.iloc[self.current_session_index]['reward'] 
            self.current_session_index += 1        
            return next_state, float(self.reward), done, meta
    
    def _metadata(self):
        session_metadata = self.current_session.iloc[0][RL_STAT_COLS]
        session_metadata['ended'] = 0
        session_metadata['incentive_index'] = 0
        return session_metadata
    
    
    def flush_episode_bins(self):
        episode_bins = self.episode_bins.copy()
        self.episode_bins = []
        return episode_bins
    
    def _calculate_next_state(self):
        
        if (self.current_session_index == self.current_session.shape[0]):
            return None, True, {}

        if self._continuing_in_session():
            return self._state(), False, {}
    
        return None, True, {}
         
    def _continuing_in_session(self):
        sim_minutes = self.current_session.iloc[self.current_session_index]['sim_minutes']
        current_session_minutes = self.current_session.iloc[self.current_session_index]['cum_session_time_raw']
        if current_session_minutes < sim_minutes:
            return True
        
        extending_session = self._probability_extending_session(current_session_minutes)
        
        return all([extending_session >= .3, extending_session <= .7])
        
    
    def _probability_extending_session(self, current_session_count):
        if self.metadata['incentive_index'] == 0:
            return 0
        
        scale = max(5, int(self.metadata['session_minutes'] / 5))
        continue_session = norm(
            loc=self.metadata['incentive_time'],
            scale=scale
        ).cdf(current_session_count)
        
        return continue_session
        

    def _get_events(self, user_id, session):
        subset = self.dataset[
            (self.dataset['user_id'] == user_id) &
            (self.dataset['session_30_count_raw'] == session).copy()
        ]

        subset = subset.sort_values(by=['date_time'])
        return subset
    
    def _take_action(self, action):
        if action == 0 or self.metadata['incentive_index'] > 0:
            return
        
        current_session_index = self.current_session_index if \
            self.current_session_index != self.current_session.shape[0] else self.current_session.shape[0] - 1
        
        self.metadata['incentive_index'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
        self.metadata['incentive_time'] = self.current_session.iloc[current_session_index]['cum_session_time_raw']
        
    def _state(self):

        if self.current_session_index > self.n_sequences:
            events = self.current_session.iloc[self.current_session_index - (self.n_sequences + 1):self.current_session_index][self.out_features].values
            
        else:
            delta = min((self.n_sequences + 1)- self.current_session_index, self.n_sequences)
            zero_cat = np.zeros((delta, len(self.out_features)))
            events = self.current_session.iloc[:max(self.current_session_index, 1)][self.out_features].values
            events = np.concatenate((zero_cat, events), axis=0)
            

        return events.astype(np.float32).T

In [6]:
# %load callback
# %load callback
import pandas as pd
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat
from datetime import datetime

class DistributionCallback(BaseCallback):
    
    @classmethod
    def tensorboard_setup(cls, log_dir, log_freq):
        cls._log_dir = log_dir
        cls._log_freq = log_freq

    
    def _on_step(self) -> bool:
        if self.n_calls % self._log_freq == 0:
            dist_list = self.training_env.env_method('flush_episode_bins')
            values_to_log = [item for sublist in dist_list for item in sublist if len(sublist) > 0]

            values_df = pd.DataFrame(
                values_to_log
            )
            
            
            session_size, sim_size, session_minutes, sim_minutes, ended, reward, inc_time, inc_index = (
                values_df['session_size'].mean(),
                values_df['sim_size'].mean(),
                values_df['session_minutes'].mean(),
                values_df['sim_minutes'].mean(),
                values_df['ended'].mean(),
                values_df['reward'].mean(),
                values_df['incentive_time'].mean(),
                values_df['incentive_index'].mean()
            )
            
            size_stats = {
                'session_size': session_size,
                'sim_size': sim_size,
                'ended': ended,
                'inc_index': inc_index
            }
            
            
            time_stats = {
                'session_minutes': session_minutes,
                'sim_minutes': sim_minutes,
                'reward': reward,
                'inc_time': inc_time   
            }
            
            for key, value in size_stats.items():
                self.logger.record(f'size/{key}', value)
            
            for key, value in time_stats.items():
                self.logger.record(f'time/{key}', value)

            
        return True

In [7]:
# %load policies/cnn_policy
# %load policies/cnn_policy
from typing import Dict, List, Type, Union

import gym
import torch
from gym import spaces
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import DQNPolicy
from torch import nn


class CustomConv1dFeatures(BaseFeaturesExtractor):
    
    @classmethod
    def setup_sequences_features(cls, n_sequences, n_features):
        cls.n_sequences = n_sequences
        cls.n_features = n_features
        
    
    def __init__(self, observation_space: spaces.Box, features_dim=20):
        super().__init__(observation_space, features_dim)
        
        
        self.cnn_1 = nn.Sequential(
            nn.Conv1d(self.n_features, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            
            nn.AvgPool1d(2)
        )
        
        self.cnn_2 = nn.Sequential(
            nn.Conv1d(self.n_features*2, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU()
        )
        
        self.act = nn.Sequential(
            nn.MaxPool1d(2),
            nn.Flatten(),
        )
        
        with torch.no_grad():
            out_shape = self.act(self.cnn_2(self.cnn_1(torch.zeros((1, self.n_features, self.n_sequences))))).shape[1]
            self.linear = nn.Linear(out_shape, features_dim)
    
    def forward(self, obs):
        out = self.cnn_1(obs)
        out = self.cnn_2(out)
        out = self.act(out)
        return self.linear(out)


        

In [8]:
# %load rl_util
import os
import logging
global logger

logger = logging.getLogger(__name__)
import numpy as np
from pqdm.threads import pqdm
import pdb

def download_dataset_from_s3(client, base_read_path, full_read_path):
    logger.info(f'Downloading data from {base_read_path}')
    os.makedirs(base_read_path, exist_ok=True)
    
    logger.info(f'Downloading data from dissertation-data-dmiller/{full_read_path}')
    client.download_file(
        'dissertation-data-dmiller',
        full_read_path,
        full_read_path
    )
    logger.info(f'Downloaded data from dissertation-data-dmiller/{full_read_path}')
    

def _parralel_partition_users(unique_sessions, df, index, vec_df_path):
    subset_session = df.merge(unique_sessions, on=['user_id', 'session_30_count_raw'], how='inner')
    subset_session.to_parquet(f'{vec_df_path}/batch_{index}.parquet')
    return subset_session.copy().reset_index(drop=True)
    

def batch_environments_for_vectorization(df, n_envs, vec_df_path):
   
    df[['user_id', 'session_30_count_raw']] = df[['user_id', 'session_30_count_raw']].astype(int)
   
    unique_sessions = df[['user_id', 'session_30_count_raw']].drop_duplicates().sample(frac=1).reset_index(drop=True)
    logger.info(f'Unique sessions shape: {unique_sessions.shape}. Splitting into {n_envs} environments')
    unique_session_split = np.array_split(unique_sessions, n_envs)
    
    unique_session_args = [{
        'unique_sessions': sess,
        'df': df,
        'index': i,
        'vec_df_path': vec_df_path,
    } for i, sess in enumerate(unique_session_split)]

    logger.info(f'Environments split: running parralel partitioning')
    result = pqdm(unique_session_args, _parralel_partition_users, n_jobs=os.cpu_count() * 2, argument_type='kwargs')
    logger.info(f'Environments split: finished parralel partitioning')
    return result
        
    

In [9]:
# %load incentive_reinforcement_learning_cpu.py
import argparse
import logging
import os
from datetime import datetime
from functools import reduce
from pprint import pformat
from typing import Callable
import boto3
import random
import numpy as np
import pandas as pd
import torch


from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.callbacks import (CallbackList,
                                                CheckpointCallback,
                                                StopTrainingOnMaxEpisodes)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.dqn.policies import DQNPolicy


logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
np.set_printoptions(precision=4, linewidth=200, suppress=True)
torch.set_printoptions(precision=2, linewidth=200, sci_mode=False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
global logger
logger = logging.getLogger('rl_exp_train')
logger.setLevel(logging.INFO)

S3_BASELINE_PATH = 's3://dissertation-data-dmiller/'
N_SEQUENCES = 40
CHECKPOINT_FREQ = 250_000
TB_LOG = 10_000
WINDOW = 1

def parse_args():
    parse = argparse.ArgumentParser()
    parse.add_argument('--read_path', type=str, default='rl_ready_data_conv')
    parse.add_argument('--n_files', type=int, default=2)
    parse.add_argument('--n_episodes', type=int, default=50)
    parse.add_argument('--n_envs', type=int, default=100)
    parse.add_argument('--lstm', type=str, default='label')
    parse.add_argument('--part', type=str, default='train')
    parse.add_argument('--feature_extractor', type=str, default='cnn') 
    args = parse.parse_args()
    return args


def load_and_dedupe(read_path, cols):
    
    df = pd.read_parquet(read_path, columns=cols)
    return df



def main(args):
    
    
    logger.info('Starting Incentive Reinforcement Learning')
    logger.info(pformat(args.__dict__))
    exec_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    
    read_path, n_files, n_episodes, n_envs, lstm, part, feature_ext = (
        args.read_path, 
        args.n_files, 
        args.n_episodes, 
        args.n_envs,
        args.lstm,
        args.part,
        args.feature_extractor,
    )

    base_read_path = os.path.join('rl_ready_data_conv', f'files_used_{n_files}')
    full_read_path = os.path.join(base_read_path, f'window_{WINDOW}_{part}_final.parquet')
    vec_df_path = os.path.join(base_read_path, f'window_{WINDOW}_{part}_batched')
    load_cols = LOAD_COLS

    base_exp_path = '/'.join(
        [
            'rl_experiments',
            f'n_files_{n_files}',
            f'{feature_ext}_{lstm}',
            exec_time,
        ]
    )

    if not os.path.exists(full_read_path):
        client = boto3.client('s3')
        download_dataset_from_s3(client,  base_read_path, full_read_path)
        
    if not os.path.exists(vec_df_path):
        df = load_and_dedupe(full_read_path, cols=load_cols)
        df = df.sort_values(['date_time', 'user_id'])
        logger.info(f'Loaded data with shape {df.shape}')
        os.makedirs(vec_df_path, exist_ok=True)
        logger.info(f'Writing vectorized data to {vec_df_path}')
        batch_environments_for_vectorization(df, n_envs, vec_df_path)
        logger.info(f'Vectorized environments created')
        del df
    

    logger.info(f'Loading vectorized data from {vec_df_path}')
    vectorized_df = [
        pd.read_parquet(os.path.join(vec_df_path, f'batch_{i}.parquet'), columns=LOAD_COLS)
        for i in range(n_envs)
    ]

    out_features = FEATURE_COLUMNS + [lstm] if lstm else FEATURE_COLUMNS


    citizen_science_vec =DummyVecEnv([lambda: CitizenScienceEnv(vec_df, out_features, N_SEQUENCES) for vec_df in vectorized_df])
    monitor_train = VecMonitor(citizen_science_vec)
    
    logger.info(f'Vectorized environments created')



    tensorboard_dir, checkpoint_dir = (
        os.path.join(base_exp_path, 'training_metrics'),
        os.path.join(base_exp_path, 'checkpoints')
    )

    if not os.path.exists(tensorboard_dir):
        logger.info(f'Creating directory {tensorboard_dir} for tensorboard logs')
        os.makedirs(tensorboard_dir)
   
    if not os.path.exists(checkpoint_dir):
        logger.info(f'Creating directory {checkpoint_dir} for checkpoints')
        os.makedirs(checkpoint_dir) 

    callback_max_episodes = StopTrainingOnMaxEpisodes(max_episodes=n_episodes, verbose=1)
    checkpoint_freq = int(CHECKPOINT_FREQ // n_envs)
    log_freq = int(TB_LOG // n_envs)
    checkpoint_callback = CheckpointCallback(
        save_freq=checkpoint_freq,
        save_path=checkpoint_dir, 
        verbose=2
    )
    
    DistributionCallback.tensorboard_setup(tensorboard_dir, log_freq)
    logger_callback = DistributionCallback()
    
    callback_list = CallbackList([checkpoint_callback, logger_callback, callback_max_episodes])
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    if feature_ext == 'cnn':
        CustomConv1dFeatures.setup_sequences_features(N_SEQUENCES + 1, len(out_features))
        logger.info('Using custom 1 dimensional CNN feature extractor')
        policy_kwargs = dict(
            features_extractor_class=CustomConv1dFeatures,
            net_arch=[10]
        )
        model = DQN(policy='CnnPolicy', env=monitor_train, verbose=1, tensorboard_log=tensorboard_dir, policy_kwargs=policy_kwargs, device=device, stats_window_size=1000)
    else:
        logger.info('Using default MLP feature extractor')
        model = DQN(policy='MlpPolicy', env=monitor_train, verbose=1, tensorboard_log=tensorboard_dir, device=device, stats_window_size=1000)
        
    logger.info(f'Model created: policy')
    
    logger.info(pformat(model.policy))
        
    logger.info(f'Beginning training') 
    
            
    logger.info(pformat([
        'n_episodes: {}'.format(n_episodes),
        'read_path: {}'.format(read_path),
        'n_files: {}'.format(n_files),
        'n_sequences: {}'.format(N_SEQUENCES),
        'n_envs: {}'.format(n_envs),
        'device: {}'.format(device),
        'lstm: {}'.format(lstm),
        'part: {}'.format(part),
        'feature_extractor: {}'.format(feature_ext),
        'tensorboard_dir: {}'.format(tensorboard_dir),
        'checkpoint_dir: {}'.format(checkpoint_dir),
        'checkpoint_freq: {}'.format(checkpoint_freq),
        'tb_freq: {}'.format(log_freq),
    ]))
    
    model.learn(total_timesteps=25_000_000, progress_bar=True, log_interval=log_freq, callback=callback_list)
    



In [10]:
class Argument:
    read_path = 'rl_ready_data_conv'
    n_files = 30
    n_episodes = 500_000
    n_envs = 100
    lstm = None
    part = 'train'
    feature_extractor = 'cnn'

In [11]:

main(Argument)

06/03/2023 07:33:59 PM Starting Incentive Reinforcement Learning
06/03/2023 07:33:59 PM mappingproxy({'__dict__': <attribute '__dict__' of 'Argument' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Argument' objects>,
              'feature_extractor': 'cnn',
              'lstm': None,
              'n_envs': 100,
              'n_episodes': 500000,
              'n_files': 30,
              'part': 'train',
              'read_path': 'rl_ready_data_conv'})
06/03/2023 07:33:59 PM Loading vectorized data from rl_ready_data_conv/files_used_30/window_1_train_batched
06/03/2023 07:34:00 PM Vectorized environments created
06/03/2023 07:34:00 PM Creating directory rl_experiments/n_files_30/cnn_None/2023-06-03_19-33-59/training_metrics for tensorboard logs
06/03/2023 07:34:00 PM Creating directory rl_experiments/n_files_30/cnn_None/2023-06-03_19-33-59/checkpoints for checkpoints
06/03/2023 07:34:00 PM 

Using cuda device


06/03/2023 07:34:01 PM Model created: policy
06/03/2023 07:34:01 PM CnnPolicy(
  (q_net): QNetwork(
    (features_extractor): CustomConv1dFeatures(
      (cnn_1): Sequential(
        (0): Conv1d(20, 40, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(1,))
        (7): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): Conv1d(40, 40, kernel_size=(3,), stride=(1,), padding=(1,))
        (9): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
      )
      (cnn_2): Sequential(
        (0): Conv1d(40, 20, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(20, eps=1e-05

Logging to rl_experiments/n_files_30/cnn_None/2023-06-03_19-33-59/training_metrics/DQN_1


Output()

...

-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 39        |
|    ep_rew_mean      | 58.937786 |
|    exploration_rate | 0.947     |
| time/               |           |
|    episodes         | 3600      |
|    fps              | 922       |
|    time_elapsed     | 151       |
|    total_timesteps  | 139900    |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 0.865     |
|    n_updates        | 224       |
-----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 38.8     |
|    ep_rew_mean      | 58.77157 |
|    exploration_rate | 0.945    |
| size/               |          |
|    ended            | 38.2     |
|    inc_index        | 1.95     |
|    session_size     | 54       |
|    sim_size         | 38.2     |
| time/               |          |
|    episodes         | 3700     |
|    fps              | 922      |
|    inc_time         | 1.94     |
|    reward           | 2.11     |
|    session_minutes  | 85.2     |
|    sim_minutes      | 59.5     |
|    time_elapsed     | 155      |
|    total_timesteps  | 143700   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.68     |
|    n_updates        | 234      |
----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 38.7     |
|    ep_rew_mean      | 58.90962 |
|    exploration_rate | 0.944    |
| time/               |          |
|    episodes         | 3800     |
|    fps              | 921      |
|    time_elapsed     | 159      |
|    total_timesteps  | 147000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.56     |
|    n_updates        | 242      |
----------------------------------


-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 38.8      |
|    ep_rew_mean      | 58.846134 |
|    exploration_rate | 0.942     |
| size/               |           |
|    ended            | 36.2      |
|    inc_index        | 2.07      |
|    session_size     | 51.3      |
|    sim_size         | 36.2      |
| time/               |           |
|    episodes         | 3900      |
|    fps              | 923       |
|    inc_time         | 2.61      |
|    reward           | 2.19      |
|    session_minutes  | 80.4      |
|    sim_minutes      | 56.9      |
|    time_elapsed     | 164       |
|    total_timesteps  | 151500    |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 1.84      |
|    n_updates        | 253       |
-----------------------------------


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 38.3     |
|    ep_rew_mean      | 58.46743 |
|    exploration_rate | 0.941    |
| time/               |          |
|    episodes         | 4000     |
|    fps              | 923      |
|    time_elapsed     | 167      |
|    total_timesteps  | 155000   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.17     |
|    n_updates        | 262      |
----------------------------------
